# TinyAleph: Getting Started

Welcome to TinyAleph! This notebook introduces the core concepts of the library:

- **Prime Hilbert Space (H_P)**: Quantum-like states where basis vectors are prime numbers
- **Superposition and Measurement**: Creating and collapsing quantum states
- **Entropy and Coherence**: Measuring uncertainty and focus
- **Operators**: Transforming quantum states

Let's begin!

## 1. Setup

First, let's import the necessary modules:

In [ ]:
import sys
sys.path.insert(0, '..')

from tinyaleph.hilbert.state import PrimeState
from tinyaleph.core.complex import Complex
from tinyaleph.core.primes import is_prime, nth_prime, first_n_primes, factorize
from tinyaleph.core.constants import PHI, COHERENCE_THRESHOLD, ENTROPY_THRESHOLD
import math

print(f"Golden Ratio (φ): {PHI:.6f}")
print(f"Coherence Threshold (1/φ): {COHERENCE_THRESHOLD:.6f}")
print(f"Entropy Threshold (ln(φ)): {ENTROPY_THRESHOLD:.6f}")

## 2. Prime Number Utilities

TinyAleph provides utilities for working with prime numbers:

In [ ]:
# Check if numbers are prime
test_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for n in test_numbers:
    print(f"is_prime({n}) = {is_prime(n)}")

In [ ]:
# Get the n-th prime number
for i in range(1, 11):
    print(f"The {i}-th prime is {nth_prime(i)}")

In [ ]:
# Get the first N primes
primes = first_n_primes(20)
print(f"First 20 primes: {primes}")

In [ ]:
# Factorize composite numbers
composites = [12, 42, 100, 360]
for n in composites:
    factors = factorize(n)
    factor_str = ' × '.join(f"{p}^{e}" if e > 1 else str(p) for p, e in factors.items())
    print(f"{n} = {factor_str}")

## 3. Creating Prime States

The **Prime Hilbert Space** H_P is defined as:

$$H_P = \{|\psi\rangle = \sum_p \alpha_p|p\rangle : \sum|\alpha_p|^2 = 1, \alpha_p \in \mathbb{C}, p \text{ prime}\}$$

Each basis state $|p\rangle$ corresponds to a prime number p.

In [ ]:
# Create a single prime eigenstate |7⟩
state_7 = PrimeState.basis(7)
print(f"Eigenstate |7⟩:")
print(f"  State: {state_7}")
print(f"  Primes in basis: {state_7.primes}")
print(f"  Norm: {state_7.norm():.4f}")

In [ ]:
# Create eigenstates for different primes
for p in [2, 3, 5, 11, 13]:
    state = PrimeState.single_prime(p)
    print(f"|{p}⟩: probabilities = {state.probabilities()}")

## 4. Superposition States

A superposition is a linear combination of basis states. The **uniform superposition** has equal probability for each prime:

In [ ]:
# Create uniform superposition of first 5 primes
uniform = PrimeState.first_n_superposition(5)
print(f"Uniform superposition of first 5 primes:")
print(f"  {uniform}")
print(f"\nProbabilities:")
probs = uniform.probabilities()
for p, prob in probs.items():
    print(f"  P(|{p}⟩) = {prob:.4f} ({prob*100:.1f}%)")
print(f"\nSum of probabilities: {sum(probs.values()):.4f}")

In [ ]:
# Create superposition with custom primes
custom_primes = [2, 7, 11, 23, 31]
custom_superposition = PrimeState.uniform_superposition(custom_primes)
print(f"Custom superposition:")
print(f"  Primes: {custom_superposition.primes}")
print(f"  Probabilities: {custom_superposition.probabilities()}")

In [ ]:
# Create state with custom amplitudes
custom = PrimeState([2, 3, 5])
custom.set(2, Complex(0.6, 0))   # 36% probability
custom.set(3, Complex(0.6, 0))   # 36% probability
custom.set(5, Complex(0.5292, 0))  # ~28% probability

print(f"Before normalization:")
print(f"  Norm = {custom.norm():.4f}")

# Normalize to ensure probabilities sum to 1
normalized = custom.normalize()
print(f"\nAfter normalization:")
print(f"  Norm = {normalized.norm():.4f}")
print(f"  Probabilities: {normalized.probabilities()}")

## 5. Entropy and Coherence

**Shannon Entropy** measures the uncertainty in a state:

$$H = -\sum_p |\alpha_p|^2 \log_2 |\alpha_p|^2$$

- Zero entropy: Perfect certainty (single prime eigenstate)
- Maximum entropy: Complete uncertainty (uniform superposition)

**Coherence** is inversely related to entropy - high coherence means the state is "focused."

In [ ]:
# Compare entropy of different states
eigenstate = PrimeState.basis(7)
small_super = PrimeState.first_n_superposition(3)
large_super = PrimeState.first_n_superposition(10)

print(f"{'State':<30} {'Entropy':>10} {'Max Entropy':>12} {'Coherence':>10}")
print("-" * 65)

for name, state, n in [
    ("Eigenstate |7⟩", eigenstate, 1),
    ("Superposition (3 primes)", small_super, 3),
    ("Superposition (10 primes)", large_super, 10),
]:
    entropy = state.entropy()
    max_entropy = math.log2(n) if n > 1 else 0
    coherence = state.coherence()
    print(f"{name:<30} {entropy:>10.4f} {max_entropy:>12.4f} {coherence:>10.4f}")

In [ ]:
# Visualize entropy vs number of primes
import matplotlib.pyplot as plt

n_values = list(range(1, 21))
entropies = []
max_entropies = []

for n in n_values:
    if n == 1:
        state = PrimeState.basis(2)
    else:
        state = PrimeState.first_n_superposition(n)
    entropies.append(state.entropy())
    max_entropies.append(math.log2(n) if n > 1 else 0)

plt.figure(figsize=(10, 6))
plt.plot(n_values, entropies, 'b-o', label='Uniform Superposition', linewidth=2, markersize=6)
plt.plot(n_values, max_entropies, 'r--', label='Maximum Entropy (log₂ n)', linewidth=2)
plt.axhline(y=ENTROPY_THRESHOLD, color='green', linestyle=':', label=f'Stability Threshold (ln φ ≈ {ENTROPY_THRESHOLD:.3f})')
plt.xlabel('Number of Primes', fontsize=12)
plt.ylabel('Entropy (bits)', fontsize=12)
plt.title('Entropy of Prime State Superpositions', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Inner Products

The inner product $\langle\psi|\phi\rangle$ measures the overlap between two states:

$$\langle\psi|\phi\rangle = \sum_p \bar{\alpha}_p \beta_p$$

- Orthogonal states have zero inner product
- Identical states have inner product 1

In [ ]:
# Inner product examples
primes_4 = [2, 3, 5, 7]
state_2 = PrimeState(primes_4)
state_2.set(2, Complex.one())

state_7 = PrimeState(primes_4)
state_7.set(7, Complex.one())

uniform = PrimeState.uniform_superposition(primes_4)

print("Inner Products:")
print(f"  ⟨2|2⟩ = {state_2.inner_product(state_2).re:.4f} (same state)")
print(f"  ⟨2|7⟩ = {state_2.inner_product(state_7).re:.4f} (orthogonal)")
print(f"  ⟨7|7⟩ = {state_7.inner_product(state_7).re:.4f} (same state)")
print(f"\n  ⟨2|uniform⟩ = {state_2.inner_product(uniform).norm():.4f}")
print(f"  Expected: 1/√4 = {1/math.sqrt(4):.4f}")

## 7. Measurement (Collapse)

**Measurement** collapses a superposition to a single eigenstate. The probability of measuring prime $p$ is $|\alpha_p|^2$.

In [ ]:
# Measure a uniform superposition multiple times
num_measurements = 1000
outcomes = {}

for _ in range(num_measurements):
    # Create fresh state (measurement modifies the state)
    state = PrimeState.first_n_superposition(5)
    prime, prob = state.measure()
    outcomes[prime] = outcomes.get(prime, 0) + 1

print(f"Measurement results ({num_measurements} trials):")
print(f"\n{'Prime':>6} {'Count':>8} {'Frequency':>10} {'Expected':>10}")
print("-" * 40)
for p in sorted(outcomes.keys()):
    count = outcomes[p]
    freq = count / num_measurements
    expected = 0.2  # 1/5 for 5 primes
    print(f"{p:>6} {count:>8} {freq:>10.4f} {expected:>10.4f}")

In [ ]:
# Visualize measurement statistics
plt.figure(figsize=(10, 6))
primes = sorted(outcomes.keys())
frequencies = [outcomes[p] / num_measurements for p in primes]
expected = [0.2] * len(primes)

x = range(len(primes))
width = 0.35

plt.bar([i - width/2 for i in x], frequencies, width, label='Observed', color='steelblue')
plt.bar([i + width/2 for i in x], expected, width, label='Expected', color='coral', alpha=0.7)

plt.xlabel('Prime Number', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Measurement Statistics ({num_measurements} trials)', fontsize=14)
plt.xticks(x, primes)
plt.legend()
plt.grid(True, axis='y', alpha=0.3)
plt.show()

## 8. State Arithmetic

States can be added and scaled to create new superpositions:

In [ ]:
# Create states for arithmetic
primes_small = [2, 3, 5, 7]

s2 = PrimeState(primes_small)
s2.set(2, Complex.one())

s7 = PrimeState(primes_small)
s7.set(7, Complex.one())

print("Original states:")
print(f"  |2⟩: probabilities = {s2.probabilities()}")
print(f"  |7⟩: probabilities = {s7.probabilities()}")

In [ ]:
# Addition creates superposition
combined = s2 + s7
print(f"|2⟩ + |7⟩ (before normalization):")
print(f"  Norm = {combined.norm():.4f}")

combined_norm = combined.normalize()
print(f"\nAfter normalization:")
print(f"  Probabilities = {combined_norm.probabilities()}")
print(f"  (Equal superposition of |2⟩ and |7⟩)")

In [ ]:
# Scalar multiplication
scaled = s2 * 0.5
print(f"0.5 × |2⟩:")
print(f"  Norm = {scaled.norm():.4f}")
print(f"  (Amplitude is halved)")

## 9. Complex Numbers

TinyAleph includes a pure-Python complex number implementation:

In [ ]:
# Complex number basics
z1 = Complex(3, 4)
z2 = Complex(1, -2)

print(f"z1 = {z1}")
print(f"z2 = {z2}")
print(f"\nArithmetic:")
print(f"  z1 + z2 = {z1 + z2}")
print(f"  z1 - z2 = {z1 - z2}")
print(f"  z1 × z2 = {z1 * z2}")
print(f"  z1 ÷ z2 = {z1 / z2}")

In [ ]:
# Properties
z = Complex(3, 4)
print(f"z = {z}")
print(f"\nProperties:")
print(f"  |z| (norm) = {z.norm():.4f}")
print(f"  z̄ (conjugate) = {z.conjugate()}")
print(f"  arg(z) (phase) = {z.phase():.4f} radians = {math.degrees(z.phase()):.2f}°")
print(f"  z⁻¹ (inverse) = {z.inverse()}")

In [ ]:
# Polar form
r = 2.0
theta = math.pi / 4  # 45 degrees

z_polar = Complex.from_polar(r, theta)
print(f"From polar: r={r}, θ={math.degrees(theta)}°")
print(f"  z = {z_polar}")
print(f"  = {r} × (cos({math.degrees(theta)}°) + i sin({math.degrees(theta)}°))")

## 10. States from Composite Numbers

We can create states weighted by the prime factors of a composite number:

In [ ]:
# State from composite number
composite_numbers = [12, 30, 100, 210]

for n in composite_numbers:
    factors = factorize(n)
    state = PrimeState.composite(n)
    probs = state.probabilities()
    
    factor_str = ' × '.join(f"{p}^{e}" if e > 1 else str(p) for p, e in factors.items())
    print(f"\n{n} = {factor_str}")
    print(f"  State probabilities:")
    for p, prob in sorted(probs.items()):
        if prob > 0.001:
            print(f"    P(|{p}⟩) = {prob:.4f}")

## Summary

In this notebook, we covered:

1. **Prime utilities**: `is_prime`, `nth_prime`, `first_n_primes`, `factorize`
2. **Prime states**: `PrimeState.basis()`, `PrimeState.single_prime()`, `PrimeState.uniform_superposition()`
3. **Superpositions**: Creating states as linear combinations of prime eigenstates
4. **Entropy and coherence**: Measuring uncertainty and focus
5. **Inner products**: Measuring overlap between states
6. **Measurement**: Collapsing superpositions to eigenstates
7. **State arithmetic**: Addition and scalar multiplication
8. **Complex numbers**: Pure-Python implementation
9. **Composite states**: States weighted by prime factors

**Next:** Try the quaternion notebook to explore 3D rotations!